In [ ]:
import yaml
import db
from datetime import datetime, timedelta

In [ ]:
# Carregar configurações do arquivo YAML
with open("config.yml", "r") as config_file:
    config = yaml.safe_load(config_file)

# Obter configurações do SQLite
db_config = config["sqlite"]
database_path = db_config["database_path"]
timeout = db_config.get("timeout", 5)  # Valor padrão de 5 segundos

In [ ]:
data_inicial = '2020-01-01'
data_final = '2020-12-31'

#procurar a data de pregão mais próxima.
consulta_sql = f"""SELECT * FROM datas_pregao WHERE DATA BETWEEN ? AND ?"""
resultados = db.consultar_banco(database_path, consulta_sql, (data_inicial, data_final))

# Ajusta as datas inicial e final para o intervalo de datas encontrado

datas_pregao =[resultado[0] for resultado in resultados]

if datas_pregao:
    data_inicial = datas_pregao[0]
    data_final = datas_pregao[-1]
    print(f"Data inicial: {data_inicial}")
    print(f"Data final: {data_final}")
else:
    print("Nenhuma data de pregão encontrada.")



In [ ]:
#procurar datas de vencimentos mais próximas.

def somar_dias(data_inicial, numero_dias):
    # Converte a string de entrada para um objeto de data
    data = datetime.strptime(data_inicial, "%Y-%m-%d")
    # Adiciona os dias à data
    nova_data = data + timedelta(days=numero_dias)
    # Retorna a nova data como string no mesmo formato
    return nova_data.strftime("%Y-%m-%d")

data_inicial_opcao = somar_dias(data_inicial, 30)

consulta_sql = f"""SELECT DATA FROM datas_vencimento WHERE DATA > ?"""
resultado_consulta = db.consultar_banco(database_path, consulta_sql, [data_inicial_opcao])

datas_vencimento_opcoes = [resultado[0] for resultado in resultado_consulta]
if datas_vencimento_opcoes:
    data_inicial_opcao = datas_vencimento_opcoes[0]
    print(f"Data inicial para opções: {data_inicial_opcao}")
else:
    print("Nenhuma data de vencimento encontrada após {data_inicial_opcao}.")


In [ ]:
def preco_spot_bova11(data_pregao):
    # Consulta o preço spot do BOVA11 para a data de pregão especificada
    consulta_sql = f"""SELECT PREULT FROM cotacoes_bova11 WHERE DTPREG = ?"""
    resultados = db.consultar_banco(database_path, consulta_sql, [data_pregao])
    preco_spot = resultados[0][0] if resultados else None
    return preco_spot

In [ ]:
preco_spot = preco_spot_bova11(data_inicial)
strike_inicial = int(preco_spot + 5)
print(f"Preço spot do BOVA11: {preco_spot}")
print(f"Strike inicial: {strike_inicial}")

In [ ]:
# encontrar opcao mais próxima do strike
consulta_sql = f"""SELECT DISTINCT CODNEG, PREEXE FROM opcoes_bova11 WHERE DATVEN = ? AND PREEXE = ?"""

opcao_encontrada_codneg = False
opcao_encontrada_strike = False

for strike in range(strike_inicial, strike_inicial + 2):
    resultados = db.consultar_banco(database_path, consulta_sql, (data_inicial_opcao, strike))
    if resultados:
        opcao_encontrada_codneg = resultados[0][0]
        opcao_encontrada_strike = resultados[0][1]
        print(f"Opção encontrada {opcao_encontrada_codneg} Strike {opcao_encontrada_strike}")
        break




In [ ]:
for resultado in resultados:
    print(f"Opção encontrada: {resultado}")

In [ ]:
# Exemplo de uso da classe
from portfolio import Portfolio
portfolio = Portfolio()



In [ ]:
# Adicionando posições
portfolio.adiciona_posicao("2025-03-28", "BOVAD125", "2025-04-17", -1000, 4.5)
portfolio.adiciona_posicao("2025-03-28", "BOVAD127", "2025-04-17", -1000, 2.5)
portfolio.adiciona_posicao("2025-03-28", "BOVAD129", "2025-04-17", -1000, 1.5)
portfolio.historico[("BOVAD125", "2025-04-17")]

In [ ]:
portfolio.copia_posicoes('2025-03-28', '2025-04-29')


In [ ]:
portfolio.posicoes('2025-04-29')

In [ ]:
portfolio.remove_posicao("2025-03-28", "BOVAD125", "2025-04-17", 2.5)

In [ ]:
portfolio.historico[("BOVAD125", "2025-04-17")]

In [ ]:
portfolio.historico_valor()

In [ ]:
portfolio.posicoes_fechadas()